In [31]:
import aocd
data = aocd.get_data(day=1, year=2025)
data = [ int(v[1:]) if v[0] == "R" else -int(v[1:]) for v in data.splitlines()]
part1, part2, cur = 0, 0, 50
for v in data:
    if v>0:
        md, cur = divmod(cur+v,100)
        part2 += abs(md)
    else:
        md, ncur = divmod((100-cur)%100-v,100)
        part2 += abs(md)
        cur = (100-ncur)%100
    part1 += cur == 0
print("part1",part1,"part2",part2)

part1 1135 part2 6558


-1